# Verify a FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See enveloped signatures: http://build.fhir.org/signatures.html

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Fetch Signed Bundle or QR from FHIR Server

In [ ]:
from requests import get
from json import dumps

# bundle_id = "f50e72a6-290e-44f8-9b6f-adf19713d0d4/_history/183"  # insert bundle id here
bundle_id = 'Bundle-cdex-searchbundle-digital-sig-example.json'

# url = "https://argopatientlist.aidbox.app/fhir/Bundle"
# url = 'http://test.fhir.org/r4/Bundle'
url = 'https://hl7.org/fhir/us/davinci-cdex'

username = "basic"
password = "secret"
headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

r = get(f'{url}/{bundle_id}', auth=(username, password), headers = headers)
my_obj = r.json()
print("="*80)
print("STATUS: ",r.status_code)

print("="*80)
print("HEADERS:\n")
for k,v in r.headers.items():
    print(f'{k} = {v}')
print("="*80)
print("BODY:\n")
print(dumps(my_obj,indent=2))

### Or Alternatively Fetch JSON From a Local File

## Receiver/Verifier Steps

###  1. Remove the Signature from the Bundle or QR

In [ ]:
def  pop_element(resource, element):
  try:
    my_element = resource.pop(element) # remove element
    return my_element 
  except KeyError:
    pass
 
if my_obj['resourceType'] == 'Bundle':
  recd_signature = pop_element(my_obj, 'signature') #for Bundles
elif my_obj['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(my_obj['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          my_obj_signature_ext = my_obj['extension'].pop(i) # remove element
          recd_signature = my_obj_signature_ext['valueSignature']
    if i == 0:
      my_obj.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')
else:
  print('Not a Bundle or QuestionnaireResponse')

recd_signature

### 2. Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS):

- Remove the id and meta elements if present before canonicalization

In [ ]:
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.
my_obj_id = pop_element(my_obj, 'id')
my_obj_meta = pop_element(my_obj, 'meta')
canonical_obj = canonicalize(my_obj)
canonical_obj, len(canonical_obj)

### 3. Transform canonicalize Bundle to a base64 format using the Base64-URL algorithm.

In [ ]:
from base64 import urlsafe_b64encode
recd_b64_canonical_obj  = urlsafe_b64encode(canonical_obj).decode()
recd_b64_canonical_obj = recd_b64_canonical_obj.replace("=","")  #remove padding since decode package doesn't use them 
recd_b64_canonical_obj

### 4. Get the base64 encoded JWS  from the `Bundle.signature.data`  element

In [ ]:
recd_b64_jws = recd_signature['data']
recd_b64_jws

### 5. Base64 decode the encoded JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [ ]:
labels = ['header', 'payload', 'signature']
from base64 import b64decode
recd_jws = b64decode(recd_b64_jws.encode()).decode()
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

### 6. reattach the payload - the base64 encoded Bundle or QR - into the JWS payload element. 

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing

In [ ]:
split_sig = recd_jws.split('.')
split_sig[1] = recd_b64_canonical_obj
recd_jws = '.'.join(split_sig)
for i,j in enumerate(recd_jws.split('.')):
    print(f'{labels[i]}:')
    print(f'{j}')
    print()

### 7. Obtain public Key from the first certificate in JWS header `"x5c"` key

- base64 decode the key value
- Verify Issuer, Validity Dates, Subject,and KeyUsage of certificate,
- Get the “Subject Public Key Info” from the cert to verify the signature

In [ ]:
from jose import jws
recd_header = jws.get_unverified_header(recd_jws)
recd_header

### 8. Inspect the x509 certificate for the

- Issuer
- Subject
- Key Usage
- Validaty dates
- SAN

In [ ]:
recd_cert = b64decode(recd_header['x5c'][0])
with open('recd_cert.der', 'wb') as f:
    f.write(recd_cert)
!openssl x509 -in recd_cert.der -inform DER -text

### 9. Verify Signature using the public key or the X.509 Certificate

Alternatively:
1. visit https://jwt.io.
1. At the top of the page, select "RS256" for the algorithm.
1. Paste the JWS value printed below into the “Encoded” field.
1. The plaintext JWT will be displayed in the “Decoded:Payload” field.
1. The X509 Cert above will appear in the "Verify Signature" box.
1. If verified, a “Signature Verified” message will appear  in the bottom left hand corner.

In [ ]:
recd_jws

In [ ]:
!openssl x509 -in recd_cert.der -inform DER -pubkey -noout > recd-public-key.pem
with open('recd-public-key.pem') as f:
    pem_public = f.read()
pem_public

In [ ]:
try:
    verify = jws.verify(recd_jws, pem_public , algorithms=['RS256'])
    print('#     #                                                ### ')
    print('#     #  ######  #####   #  ######  #  ######  #####   ### ')
    print('#     #  #       #    #  #  #       #  #       #    #  ### ')
    print('#     #  #####   #    #  #  #####   #  #####   #    #   #  ')
    print(' #   #   #       #####   #  #       #  #       #    #      ')
    print('  # #    #       #   #   #  #       #  #       #    #  ### ')
    print('   #     ######  #    #  #  #       #  ######  #####   ### ')
    print('                                                           ')

except Exception as e:
    print('#    #   ####   #####      #    #  ######  #####   #  ######  #  ######  #####       ###          #   ')
    print('##   #  #    #    #        #    #  #       #    #  #  #       #  #       #    #       #          #    ')
    print('# #  #  #    #    #        #    #  #####   #    #  #  #####   #  #####   #    #           #####  #    ')
    print('#  # #  #    #    #        #    #  #       #####   #  #       #  #       #    #       #          #    ')
    print('#   ##  #    #    #         #  #   #       #   #   #  #       #  #       #    #      ###          #   ')
    print('#    #   ####     #          ##    ######  #    #  #  #       #  ######  #####        #            ## ')
    print('                ')
    print(f"not verified: {e}")